In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataCleaningConfig:
    root_dir: Path
    columns: dict
    data_path: Path
    target_column: str
    object_columns: list

In [ ]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_cleaning_config(self) -> DataCleaningConfig:
        config = self.config.data_cleaning
        target_column = self.schema.TARGET_COLUMN
        object_columns = self.schema.OBJECT_COLUMNS

        create_directories([config.root_dir])

        data_cleaning_config = DataCleaningConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            target_column=target_column.name,
            object_columns=object_columns
        )

        return data_cleaning_config

In [ ]:
import pandas as pd
import numpy as np
import os
from src.datascience import logger

In [ ]:
class DataCleaning:
    def __init__(self, config: DataCleaningConfig):
        self.config = config
    
    def clean_data(self):
        data = pd.read_csv(self.config.data_path)
        data=pd.get_dummies(data,columns=self.config.object_columns,drop_first=True)
        data[self.config.target_column]=np.where(data[self.config.target_column]=="Y",1,0)
        data=data.dropna()
        
        data.to_csv(os.path.join(self.config.root_dir, "credit_risk_dataset.csv"),index = False)
        logger.info("Data cleaning completed successfully")
        logger.info(data.shape)
        
        print(data.shape)